# Advanced Querying Mongo

Importing libraries and setting up connection

In [5]:
from pymongo import MongoClient,DESCENDING,ASCENDING
import re

dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/companies


### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [12]:
# Your Code
query = db.companies.find({"name":"Babelgum"},{"name":1,"_id":0})
for name in query:
    print(name)

{'name': 'Babelgum'}


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [33]:

number_employees = db.companies.find({"number_of_employees":{"$gte":5000}}).sort([("number_of_employees",ASCENDING)]).limit(20)
len(list(number_employees))


20

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [57]:

years_founded = {
    "$and":[
        {"founded_year":{"$gte":2000}},
        {"founded_year":{"$lte":2005}}
    ]
}

fields={"name":1,"founded_year":1,"_id":0}
list(db.companies.find(years_founded,fields).limit(5))



[{'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [58]:
query={
    "$and":[
        {"ipo.valuation_amount":{"$gte":100000000}},
        {"founded_year":{"$lte":2010}}
    ]
}
fields={"name":1,"ipo":1,"_id":0}
list(db.companies.find(query,fields).limit(5))


[{'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'name': 'Brightcove',
  'ipo': {'valuation_amount': 290000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 2,
   'pub_day': 17,
   'stock_symbol': 'NASDAQ:BCOV'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [107]:
# Your Code
query={
    "$and":[
        {"number_of_employees":{"$gte":1000}},
        {"founded_year":{"$lte":2005}}
    ]
}
fields = {"number_of_employees":1,"_id":0}
list(db.companies.find(query,fields).limit(10))

[{'number_of_employees': 1600},
 {'number_of_employees': 5299},
 {'number_of_employees': 15000},
 {'number_of_employees': 63000},
 {'number_of_employees': 13600},
 {'number_of_employees': 86300},
 {'number_of_employees': 28000},
 {'number_of_employees': 4800},
 {'number_of_employees': 5080},
 {'number_of_employees': 1600}]

### 6. All the companies that don't include the `partners` field.

In [106]:
# Your Code
db.companies.find({"partners":{ "$exists": True}}).count()


                   

<ipython-input-106-6891d2b9619a>:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  db.companies.find({"partners":{ "$exists": True}}).count()


18801

### 7. All the companies that have a null type of value on the `category_code` field.

In [102]:
# Your Code
b = db.companies.find({"category_code":{"$exists":True}})
len(list(b))

18801

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [109]:
query={
    "$and":[
        {"number_of_employees":{"$gte":100}},
        {"number_of_employees":{"$lte":1000}}
    ]
}
fields = {"number_of_employees":1,"_id":0,"name":1}
list(db.companies.find(query,fields).limit(10))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [125]:
# Your Code
list(db.companies.find({"name":{"$exists":True}},{"_id":0,"name":1}).sort(["ipo.valuation_amount",DESCENDING]))

# Your Code


ValueError: too many values to unpack (expected 2)

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [138]:
search={"_id":0,"name":1,"number_of_employees":1}
list(db.companies.find({},search).limit(10).sort("number_of_employees",DESCENDING))


[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [143]:
# Your Code
query={"founded_month":{"$gte":5}}

fields = {"name":1,"_id":0,"founded_month":1}
list(db.companies.find(query,fields).limit(1000))[10:20]

[{'name': 'Technorati', 'founded_month': 6},
 {'name': 'eBay', 'founded_month': 9},
 {'name': 'OpenX', 'founded_month': 5},
 {'name': 'Sparter', 'founded_month': 6},
 {'name': 'Kyte', 'founded_month': 12},
 {'name': 'Thoof', 'founded_month': 12},
 {'name': 'Jingle Networks', 'founded_month': 9},
 {'name': 'LifeLock', 'founded_month': 8},
 {'name': 'Wesabe', 'founded_month': 12},
 {'name': 'SmugMug', 'founded_month': 11}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [147]:
# Your Code
query={
    "$and":[
        {"founded_year":{"$lt":2000}},
        {"acquisition.price_amount":{"$gte":10000000}}
    ]
}
fields = {"founded_year":1,"_id":0,"acquisition.price_amount":1,"name":1}
list(db.companies.find(query,fields).limit(10))

[{'name': 'Postini',
  'founded_year': 1999,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep',
  'founded_year': 1999,
  'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar',
  'founded_year': 1999,
  'acquisition': {'price_amount': 25000000}},
 {'name': 'PayPal',
  'founded_year': 1998,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Snapfish',
  'founded_year': 1999,
  'acquisition': {'price_amount': 300000000}},
 {'name': 'Neopets',
  'founded_year': 1999,
  'acquisition': {'price_amount': 160000000}},
 {'name': 'Sun Microsystems',
  'founded_year': 1982,
  'acquisition': {'price_amount': 7400000000}},
 {'name': 'Zappos',
  'founded_year': 1999,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba',
  'founded_year': 1999,
  'acquisition': {'price_amount': 1000000000}},
 {'name': 'Sabre',
  'founded_year': 1960,
  'acquisition': {'price_amount': 4300000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:
# Your Code

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [ ]:
# Your Code

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code